In [12]:
from __future__ import print_function
import datetime
print(datetime.datetime.now())
import http.server
import urllib.parse
from urllib.parse import urlparse
import matplotlib
import datetime
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt




2020-04-24 23:17:54.763534


In [13]:
#Read the CSV Data first
originalhistData = pd.read_csv("C:\\Users\\ATOM_PROD1\\OneDrive\\LADDU\\DS4A-LATAM\\Project-CREDITAS-PropertyApproval-2020\\MVP\\PreviousDecisions-v1.csv")
originalhistData.head()

Unnamed: 0 model_decision analyst_decision    zip_code  \
0           4              G                A  11661100.0   
1          27              G                A   9070310.0   
2          29              G                A   6716762.0   
3          37              G                R   4470250.0   
4          39              A              NaN  29101546.0   

                                              concat  \
0  Rua Benedito Zacarias Arouca, Arouca, Caraguat...   
1          Avenida Tietê, CAMPESTRE, Santo André, SP   
2          Rua da Kumamoto, Nakamura Park, Cotia, SP   
3  Rua Luigi Mancinelli, Jardim bandeirantes, São...   
4  Ruz dionizio abaurre, 543, Jardim camburi, Vit...   

                         coordinates  
0  (-45.40042039999999, -23.6215522)  
1         (-46.5495673, -23.6369601)  
2         (-46.9008539, -23.6023936)  
3           (-46.663085, -23.712473)  
4         (-40.2681806, -20.2507639)

In [14]:
modelData = pd.read_csv("C:\\Users\\ATOM_PROD1\\OneDrive\\LADDU\\DS4A-LATAM\\Project-CREDITAS-PropertyApproval-2020\\MVP\\model_data.csv")
modelData.head()


Gen Random Values    zip_code                        coordinates  \
0            1900000  11661100.0  (-45.40042039999999, -23.6215522)   
1            2800000   9070310.0         (-46.5495673, -23.6369601)   
2            1100000   6716762.0         (-46.9008539, -23.6023936)   
3             900000   4470250.0           (-46.663085, -23.712473)   
4            1300000  29101546.0         (-40.2681806, -20.2507639)   

   Random Loan Values  Weighted Colateral Value analyst_decision  \
0              700000                   1050000                A   
1              700000                   1050000                A   
2              800000                   1200000                A   
3             1400000                   1120000                R   
4             2900000                   4350000                A   

  model_decision                                             concat  
0              G  Rua Benedito Zacarias Arouca, Arouca, Caraguat...  
1              G        Avenida TietÃª, CAMPESTRE, Santo AndrÃ©, SP  
2              G          Rua da Kumamoto, Nakamura Park, Cotia, SP  
3              G  Rua Luigi Mancinelli, Jardim bandeirantes, SÃ£...  
4              A  Ruz dionizio abaurre, 543, Jardim camburi, Vit...

In [93]:
histData.count()

Unnamed: 0          41770
model_decision      41761
analyst_decision    39742
zip_code            41768
concat              41770
coordinates         41757
dtype: int64

In [16]:
modelData['model_decision'].fillna('NA',inplace=True)
modelData['analyst_decision'].fillna('NA',inplace=True)
modelData['zip_code'].fillna('000000',inplace=True)
modelData['coordinates'].fillna('(0,0)',inplace=True)


modelData.count()

Gen Random Values           41770
zip_code                    41770
coordinates                 41770
Random Loan Values          41770
Weighted Colateral Value    41770
analyst_decision            41770
model_decision              41770
concat                      41770
dtype: int64

In [102]:
modelData['splitcoordinates'] = modelData['coordinates'].str.replace('(','').str.replace(')','').str.split(',',1)
modelData['latitude'] = 0.1
modelData['lontitude'] = 0.1

for i,ival in modelData.iterrows():
    modelData.at[i,'latitude']   = ival['splitcoordinates'][0]
    modelData.at[i,'longtitude'] = ival['splitcoordinates'][1]

print(modelData['latitude'])
print(modelData['longtitude'])



0       -45.400420
1       -46.549567
2       -46.900854
3       -46.663085
4       -40.268181
           ...    
41765   -46.671396
41766   -46.549967
41767   -47.543763
41768   -46.549657
41769   -46.768518
Name: latitude, Length: 41770, dtype: float64
0       -23.621552
1       -23.636960
2       -23.602394
3       -23.712473
4       -20.250764
           ...    
41765   -23.455998
41766   -23.440976
41767   -22.380663
41768   -23.537900
41769   -23.641504
Name: longtitude, Length: 41770, dtype: float64


In [17]:
print('model_decision values:')
pd.Index(modelData['model_decision']).value_counts()

model_decision values:


G        38001
error     1629
A         1247
R          884
NA           9
Name: model_decision, dtype: int64

In [18]:
print('analyst_decision values:')
pd.Index(modelData['analyst_decision']).value_counts()

analyst_decision values:


R    23434
A    18336
Name: analyst_decision, dtype: int64

In [19]:
print('unique values:')
pd.Index(modelData['model_decision']+'__'+modelData['analyst_decision']).value_counts()

unique values:


G__R        21719
G__A        16282
A__A         1247
R__R          872
error__R      834
error__A      795
R__A           12
NA__R           9
dtype: int64

In [20]:
same_count = 0
diff_count = 0

accept_count = 0
reject_count = 0
undecided_count = 0

accept_states_list = []
accept_states_moreinfo = []

reject_states_list = []
reject_states_moreinfo = []

#print(histData['concat'].rsplit('/', 1)[-1])

for index, row in modelData.iterrows():
    if row['model_decision'] != row['analyst_decision']:
        diff_count = diff_count + 1     
    else:
        same_count = same_count + 1
    if row['model_decision'] == 'A' or row['analyst_decision'] == 'A':
        accept_count = accept_count + 1
        accept_states_list.append(row['concat'].rsplit(',', 1)[-1])
        accept_states_moreinfo.append(row['concat'].rsplit(',')[2:])
        
    elif row['model_decision'] == 'R' or row['analyst_decision'] == 'R':
        reject_count = reject_count + 1
        reject_states_list.append(row['concat'].rsplit(',', 1)[-1])
        reject_states_moreinfo.append(row['concat'].rsplit(',')[2:])
    else:
        undecided_count = undecided_count + 1
        
print("Total Count:%d, Diff_count:%d, Same_Count:%d" %((same_count+diff_count),diff_count, same_count))
print("Accept:%d(%d), Reject:%d, undecided:%d" %(accept_count,((accept_count/(same_count+diff_count))*100),reject_count,undecided_count))


Total Count:41770, Diff_count:39651, Same_Count:2119
Accept:18336(43), Reject:23434, undecided:0


In [21]:
accept_val_list = pd.Index(accept_states_list).value_counts()
reject_val_list = pd.Index(reject_states_list).value_counts()


In [102]:
for keyVal in accept_val_list.keys():
    if(keyVal != ' '):
        totalVal = (accept_val_list[keyVal] + reject_val_list[keyVal])
        print('State:%s A:%d(%d), R:%d' %(keyVal, accept_val_list[keyVal],((accept_val_list[keyVal]/totalVal)*100), reject_val_list[keyVal]))


State: SP A:10658(48), R:11492
State: PR A:1549(52), R:1421
State: MG A:1268(42), R:1724
State: RJ A:1007(33), R:1966
State: SC A:962(44), R:1223
State: RS A:853(41), R:1208
State: GO A:261(31), R:572
State: DF A:251(38), R:399
State: MS A:231(39), R:349
State: BA A:212(29), R:514
State: CE A:188(39), R:283
State: PE A:156(28), R:391
State: ES A:148(28), R:376
State: PB A:128(45), R:156
State: MT A:108(27), R:279
State: RN A:85(37), R:141
State: SE A:55(36), R:95
State: PA A:50(24), R:152
State: AM A:40(13), R:250
State: PI A:40(40), R:59
State: MA A:36(17), R:169
State: AL A:33(26), R:92
State: TO A:16(28), R:41


In [22]:
new_accept_list = []
new_reject_list = []

for row in accept_states_moreinfo:
    new_accept_list.append(row[0] + ',' + row[1])

for row in reject_states_moreinfo:
    new_reject_list.append(row[0] + ',' + row[1])

accept_val_list = pd.Index(new_accept_list).value_counts()
reject_val_list = pd.Index(new_reject_list).value_counts()    

for keyVal in accept_val_list.keys():
    if(keyVal != ' '):
        if((keyVal in reject_val_list) == True):
            totalVal = (accept_val_list[keyVal] + reject_val_list[keyVal])
            print('State:%s A:%d(%d), R:%d' %(keyVal, accept_val_list[keyVal],((accept_val_list[keyVal]/totalVal)*100), reject_val_list[keyVal]))

    

State: SÃ£o Paulo, SP A:3495(52), R:3206
State: Curitiba, PR A:689(61), R:427
State: Rio de Janeiro, RJ A:627(45), R:761
State: Campinas, SP A:391(53), R:340
State: Guarulhos, SP A:358(46), R:413
State: Belo Horizonte, MG A:349(43), R:453
State: Santo AndrÃ©, SP A:346(57), R:252
State: Porto Alegre, RS A:324(52), R:288
State: SÃ£o Bernardo do Campo, SP A:288(55), R:231
State: Sorocaba, SP A:250(49), R:252
State: BrasÃ­lia, DF A:229(37), R:387
State: SÃ£o JosÃ© dos Campos, SP A:226(51), R:214
State: JundiaÃ­, SP A:225(50), R:220
State: Praia Grande, SP A:215(69), R:96
State: RibeirÃ£o Preto, SP A:210(52), R:191
State: UberlÃ¢ndia, MG A:208(53), R:181
State: GoiÃ¢nia, GO A:195(44), R:244
State: Campo Grande, MS A:187(40), R:273
State: Osasco, SP A:185(37), R:310
State: Santos, SP A:175(69), R:78
State: Fortaleza, CE A:173(40), R:254
State: Londrina, PR A:171(55), R:137
State: FlorianÃ³polis, SC A:162(55), R:129
State: Joinville, SC A:153(36), R:266
State: SÃ£o JosÃ© do Rio Preto, SP A:14

In [24]:
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.optimizers import RMSprop, Adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder


import os
import gc
import tensorflow as tf
from keras import backend as K
from keras.backend.tensorflow_backend import set_session




C:\Users\ATOM_PROD1\OneDrive\atom-local\Anaconda3\envs\atom_machine\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [209]:
#Model Hyperparameteres
isFast = False
EPOCHS = 10 if isFast else 1500
BATCH_SIZE_FOR_TRAIN = 100
VERBOSE_MODE=0

#MODEL_A_Input = ZipCode,Latitude,Longtitude
MODEL_A_H_DIMS = 3
MODEL_A_T_LAG=1 #This parameter can be useful when we use
MODEL_A_NUM_FEATURES_PER_SAMPLE = (MODEL_A_H_DIMS * MODEL_A_T_LAG) #KERAS
#MODEL_A_Output = A:Accept, R:Reject
MODEL_A_NUM_OUTPUT_CLASSES=2

MODEL_OUTPUT_ACTION = ['ACCEPT','REJECT']

MODEL_OUTPUT_ENCODER_ONE = LabelEncoder()
MODEL_OUTPUT_ENCODER_ONE.fit(MODEL_OUTPUT_ACTION)
print(MODEL_OUTPUT_ACTION)
print(dict(zip(MODEL_OUTPUT_ENCODER_ONE.classes_, MODEL_OUTPUT_ENCODER_ONE.transform(MODEL_OUTPUT_ENCODER_ONE.classes_))))

MASTER_OUTPUT_ACTION = [["ACCEPT","REJECT"]]
master_encoder_All = MultiLabelBinarizer()
master_encoder_All.fit(MASTER_OUTPUT_ACTION)
print(master_encoder_All.classes_)



['ACCEPT', 'REJECT']
{'ACCEPT': 0, 'REJECT': 1}
['ACCEPT' 'REJECT']


In [135]:
complete_input_data_set = np.zeros_like(np.ndarray(shape=(modelData.shape[0] * 1, 1, MODEL_A_NUM_FEATURES_PER_SAMPLE), dtype=np.float))
complete_input_data_ctr = 0

complete_output_set = []

for index, row in modelData.iterrows():    
        np_currentt_input_data = np.zeros_like(np.ndarray(shape=(1, MODEL_A_H_DIMS), dtype=np.float))
        np.copyto(np_currentt_input_data ,np.array([pd.to_numeric(row[['zip_code','latitude','lontitude']])],dtype=np.float))        
        np_currentt_input_data = np_currentt_input_data.reshape((1,MODEL_A_H_DIMS))
    
        complete_input_data_set[complete_input_data_ctr]=np_currentt_input_data
        complete_input_data_ctr=complete_input_data_ctr+1
        
        output_decision  = ''
        if row['analyst_decision'] == 'A':
            output_decision=["ACCEPT"]
        else:
            output_decision=["REJECT"]
        
        complete_output_set.append(output_decision)
        
#complete_output_set_ENCODED = MODEL_OUTPUT_ENCODER_ONE.transform(complete_output_set)

complete_output_set_ENCODED = master_encoder_All.transform(complete_output_set)
        
#print(complete_output_set_ENCODED.shape)
#print(complete_output_set)
#print(complete_input_data_set)    
print(complete_output_set_ENCODED)  
print('OK')        


    

[[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [0 1]]
OK


In [181]:
np_complete_output_set_ENCODED = np.zeros_like(np.ndarray(shape=(1, 2), dtype=np.integer))
complete_output_set_ENCODED_array=np.array(complete_output_set_ENCODED)
print(complete_output_set_ENCODED_array.shape[0])
#np.copyto(np_complete_output_set_ENCODED,np.array(pd.to_numeric([complete_output_set_ENCODED_array]),dtype=np.integer))
np_complete_output_set_ENCODED = complete_output_set_ENCODED_array.reshape(complete_output_set_ENCODED_array.shape[0],1,complete_output_set_ENCODED_array.shape[1])
#print(np_complete_output_set_ENCODED.shape)
print(np_complete_output_set_ENCODED)


41770
[[[1 0]]

 [[1 0]]

 [[1 0]]

 ...

 [[1 0]]

 [[1 0]]

 [[0 1]]]


In [207]:
K.clear_session()
tf.reset_default_graph()

model = Sequential()
model.add(Dense(500, activation='relu', input_shape=(1,MODEL_A_H_DIMS)))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
#model.add(Flatten())
#model.add(Dense(output_dim=2, activation='softmax'))
model.add(Dense(output_dim=2,activation='softmax'))

# Compile the model
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

#model.add(Flatten())
#model.add(Dense(output_dim=output_dim_count,activation='sigmoid'))
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(complete_input_data_set, np_complete_output_set_ENCODED, test_size=0.60)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1, 500)            2000      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 100)            50100     
_________________________________________________________________
dense_3 (Dense)              (None, 1, 50)             5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1, 2)              102       
Total params: 57,252
Trainable params: 57,252
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\ATOM_PROD1\OneDrive\atom-local\Anaconda3\envs\atom_machine\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=2)`
  # Remove the CWD from sys.path while we load stuff.


In [210]:
print(X_train.shape)
print(y_train.shape)

print(datetime.datetime.now())

build_metrics = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=EPOCHS, batch_size=BATCH_SIZE_FOR_TRAIN,verbose=VERBOSE_MODE,shuffle=False)

print(datetime.datetime.now())

finalmachinename = "Creditas_Model_%0.2lf_%0.2lf_ModelA" % (build_metrics.history['acc'][len(build_metrics.history['acc'])-1], build_metrics.history['loss'][len(build_metrics.history['loss'])-1])

model_json = model.to_json()

MACHINE_PATH="C:\\Users\\ATOM_PROD1\\OneDrive\\LADDU\\DS4A-LATAM\\Project-CREDITAS-PropertyApproval-2020\\MVP"

with open(MACHINE_PATH + '\\' + finalmachinename + ".json", "w") as json_file:
    json_file.write(model_json)

model.save_weights(MACHINE_PATH + '\\' + finalmachinename + ".h5")
print("Saved model to disk")

#return finalmachinename,build_metrics.history['acc'][len(build_metrics.history['acc'])-1], build_metrics.history['loss'][len(build_metrics.history['loss'])-1]


(16708, 1, 3)
(16708, 1, 2)
2020-04-25 09:20:55.470059
2020-04-25 09:51:38.117554
Saved model to disk


In [211]:
modelFile = open(MACHINE_PATH + '\\' + finalmachinename + ".json", 'r')
model_json = modelFile.read()
modelFile.close()

modelLoaded = model_from_json(model_json)
modelLoaded.load_weights(MACHINE_PATH + '\\' + finalmachinename + ".h5")

modelLoaded.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])




In [216]:
input_For_Prediction = [11661100,-45.40042039999999, -23.6215522]
input_For_Prediction2 = [29101546,-40.2681806,-20.2507639]

inputReshape = np.array(input_For_Prediction).reshape(1,1, MODEL_A_NUM_FEATURES_PER_SAMPLE)
inputReshape2 = np.array(input_For_Prediction2).reshape(1,1, MODEL_A_NUM_FEATURES_PER_SAMPLE)

projectedVal=np.array(modelLoaded.predict(inputReshape))

print(projectedVal)

projectedVal=np.array(modelLoaded.predict(inputReshape2))

print(projectedVal)

[[[1. 0.]]]
[[[1. 0.]]]
